In [1]:
import numpy as np
np.random.seed(1)
import matplotlib.pyplot as plt
from skimage import color
from skimage import io
import pandas as pd
np.random.seed(1)
#
df1 = pd.read_csv(r"D:\Learn\BJ13_M32x32_T30_In.csv")
df2 = pd.read_csv(r"D:\Learn\BJ14_M32x32_T30_In.csv")
df3 = pd.read_csv(r"D:\Learn\BJ15_M32x32_T30_In.csv")
df4 = pd.read_csv(r"D:\Learn\BJ16_M32x32_T30_In.csv")

array1 = df1.to_numpy()
array2 = df2.to_numpy()
array3 = df3.to_numpy()
array4 = df4.to_numpy()

max_rows = max(array1.shape[0], array2.shape[0], array3.shape[0], array4.shape[0])
max_cols = max(array1.shape[1], array2.shape[1], array3.shape[1], array4.shape[1])

def pad_array(array, shape):
    padded = np.zeros(shape)
    padded[:array.shape[0], :array.shape[1]] = array
    return padded

array1_padded = pad_array(array1, (max_rows, max_cols))
array2_padded = pad_array(array2, (max_rows, max_cols))
array3_padded = pad_array(array3, (max_rows, max_cols))
array4_padded = pad_array(array4, (max_rows, max_cols))

X0 = np.stack((array1_padded, array2_padded, array3_padded, array4_padded), axis=0)
print(X0.shape)
print(type(X0))

(4, 7219, 1024)
<class 'numpy.ndarray'>


In [2]:
def dell(array1,missing_rate):
    array2 = array1 * np.round(np.random.rand(array1.shape[0], array1.shape[1]) + 0.5 - missing_rate)
    return array2
array1 = dell(array1,0.5)
array2 = dell(array2,0.5)
array3 = dell(array3,0.5)
array4 = dell(array4,0.5)

In [3]:
array1_dell = pad_array(array1, (max_rows, max_cols))
array2_dell = pad_array(array2, (max_rows, max_cols))
array3_dell = pad_array(array3, (max_rows, max_cols))
array4_dell = pad_array(array4, (max_rows, max_cols))
X = np.stack((array1_padded, array2_padded, array3_padded, array4_padded), axis=0)
print(X.shape)
print(type(X))

(4, 7219, 1024)
<class 'numpy.ndarray'>


In [4]:
print(array1.shape)
X=dell(array1,0.5)
print(X.shape)
X0=array1.reshape(4887,32,32)
X=X.reshape(4887,32,32)
X=X[:32,:,:]
print(X.shape)
X0=X0[:32,:,:]

(4887, 1024)
(4887, 1024)
(32, 32, 32)


In [41]:
import tensorly as tl
from tensorly.decomposition import tucker
def Res():
    return 10
def fold(unfolded, shape, mode):
    full_shape = np.array(shape)
    full_shape[mode], full_shape[0] = full_shape[0], full_shape[mode]
    return np.moveaxis(np.reshape(unfolded, full_shape), 0, mode)
def unfold(z,rank):
    return np.reshape(np.moveaxis(z, rank, 0), (z.shape[rank], -1))
def update_cg(var, r, q, Aq, rold):
    alpha = rold / np.inner(q, Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew / rold) * q
    return var, r, q, rnew
def ell_G(G, S, R, M, X, K,n):
    print(K.shape,X.shape,S.shape)
    return K@G@S.T@R.T@M.T@S.T@R.T@M.T@+n@G
def conj_grad_G(G, S, R, M, X, K, n,maxiter = 5):
    rank, dim1 = G.shape
    print(K.shape,X.T.shape,S.shape,R.T.shape,M.T.shape)
    r = np.reshape(K@X.T@S@R.T@M.T-ell_G(G, S, R, M, X, K,n) ,-1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_G(G, S, R, M, X, K,H,n), -1, order = 'F')
        G, r, q, rold = update_cg(G, r, q, Aq, rold)
    return np.reshape(G, (rank, dim1), order = 'F')
def conj_grad_S_R_M(G, S, R, M, X, K,Q1,H,a,b,n ,r1,maxiter = 5):
    def ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        sum1=np.zeros((32,32))
        for t in range(X.shape[2]):
            sum1+=K[t,:,:]@S@G@(np.kron(M[:,t].T,R).T)@K[t,:,:].T@np.kron(M[:,t].T,R)@G.T
            #sum+=K[t,:,:]@S
        return (sum1+r1*S-a*b*H.T*Q1.T+n*S)
    def ell_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        return (K@R@G@(np.kron(M,S)).T@(np.kron(M,S))@G.T
                +r1*(R-A*Q@R-b*(np.kron(Q1,H))@R)@(np.eye(Q1.shape[0])-a*Q1.T-b*(np.kron(Q1,H)).T)+n*R)
    def ell_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ):
        return (K@M@G@(np.kron(R,S)).T@(np.kron(R,S))@G.T
                +r1*(M-A*Q@M-b*(np.kron(Q1,H))@M)@(np.eye(Q1.shape[0])-a*Q1.T-b*(np.kron(Q1,H)).T)+n*M)

    def con_grad_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        rank, dim1 = S.shape
        s=S.reshape(-1, order = 'F')
        B=np.zeros((32,32))
        for t in range(X.shape[2]):
            B+=K[t,:,:]@X[t,:,:]@K[t,:,:].T@np.kron(M[:,t].T,R)@G.T
        print("bbbb")
        print(B.shape)
        r=np.reshape(B-ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_S(G, Q, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            s, r, q, rold = update_cg(s, r, q, Aq, rold)        
        return np.reshape(S, (rank, dim1), order = 'F')
    
    def con_grad_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        X=unfold(X,2)
        G=unfold(G,2)
        K=unfold(K,2)
        rank, dim1 = R.shape
        r=np.reshape(K@X@(np.kron(M,S))@G.T-ell_R(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            R, r, q, rold = update_cg(R, r, q, Aq, rold)        
        return np.reshape(R, (rank, dim1), order = 'F')

    def con_grad_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5):
        X=unfold(X,3)
        G=unfold(G,3)
        K=unfold(K,3)
        rank, dim1 = M.shape
        r=np.reshape(K@X@(np.kron(R,S))@G.T-ell_M(G, S, R, M, X, K,Q1,H,a,b,n,r1) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_S(G, S, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            M, r, q, rold = update_cg(M, r, q, Aq, rold)        
        return np.reshape(M, (rank, dim1), order = 'F')
    S=con_grad_S(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)
    print(S)
    R=con_grad_R(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)                                                                                       
    M=con_grad_M(G, S, R, M, X, K,Q1,H,a,b,n,r1 ,maxiter = 5)                                                                                       
    return S,R,M

                                                                                           
                                                                                           
                                                                                           
def conj_grad_Q_H(S,R,M,Q1, H, a, b,maxiter = 5):
    def ell_Q1(Z,Q1, H, a, b):
        return (a*a*Q@Z@Z+a*b*Q@Z@np.kron(Z.T,H.T)+a*b*np.kron(Q,H)@Z@Z+b*b*np.kron(Q,H)@Z@np.kron(Z.T,H.T))
    def ell_H(Z,Q1, H, a, b):
        return (b*b*np.kron(Q,H)@Z@np.kron(Q.T,Z.T))
    def conj_grad_Q(Q1, H, S,R,M, a, b,maxiter = 5):
        rank, dim1 = Q1.shape
        Z=S
        r=np.reshape(a*Z@Z@+b*Z@(np.kron(Z.T,H.T))-ell_Q1(Z,Q1, H, a, b) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_Q1(Z,Q1, H, a, b), -1, order = 'F')
            Q1, r, q, rold = update_cg(Q1, r, q, Aq, rold)
        return np.reshape(Q1, (rank, dim1), order = 'F')

    def conj_grad_H(Q1, H, Z, a, b,maxiter = 5):
        rank, dim1 = H.shape
        Z=S
        r=np.reshape(b*Z@np.kron(Q.T,Z.T)-a*b*Q@Z@np.kron(Q.T,Z.T)-ell_H(S,R,M,Q1, H, a, b) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_H(G, S, R, M, X, K,Q1,H,a,b,n,r1), -1, order = 'F')
            H, r, q, rold = update_cg(H, r, q, Aq, rold)
        return np.reshape(H, (rank, dim1), order = 'F')
    Q1=conj_grad_Q(Q1, H, S,R,M, a, b)
    H=conj_grad_H(Q1, H, S,R,M, a, b)
    return Q1,H           
def conj_grad_a_b(Q, H, Z, a, b,maxiter = 5):
    def ell_a(Q, H, Z, a, b):
        return (Z@Q@Z@Q@Z)
    def ell_b(Q, H, Z, a, b):
        return (b*np.kron(Q,H)@Z@np.kron(Q,H)@Z)
    def conj_grad_a(Q, H, Z, a, b,maxiter = 5):
        
        r=np.reshape(Z@Q@Z-b*np.kron(Q,H)@Z@Q@Z-ell_a(Q, H, Z, a, b) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_a(Q, H, Z, a, b), -1, order = 'F')
            a, r, q, rold = update_cg(a, r, q, Aq, rold)
        return a
    def conj_grad_b(Q, H, Z, a, b,maxiter = 5):
        r=np.reshape(Z@np.kron(Q,H)@Z-a*Q@Z@np.kron(Q,H)@Z-ell_b(Q, H, Z, a, b) ,-1, order = 'F')
        q = r.copy()
        rold = np.inner(r, r)
        for it in range(maxiter):
            Q = np.reshape(q, (rank, dim1), order = 'F')
            Aq = np.reshape(ell_b(Q, H, Z, a, b), -1, order = 'F')
            b, r, q, rold = update_cg(b, r, q, Aq, rold)
        return b
    a=conj_grad_a(Q, H, Z, a, b)
    b=conj_grad_b(Q, H, Z, a, b)
    return a,b
def Star(X0,X,rank,n,r,rthers,maxiter):
    #Create K
    K = np.zeros(X.shape) 
    K[(X0 != 0) & (X == 0)] = 1
    print(K.shape)
    #Initialize G S R M Q H A B randomly
    G = np.random.rand(rank, rank, rank)
    S = np.random.rand(X.shape[2], rank)
    R = np.random.rand(X.shape[1], rank)
    M = np.random.rand(rank,X.shape[0])
    print(G.shape,S.shape,R.shape,M.shape)
    Q = np.random.rand(X.shape[0], X.shape[0])
    H = np.random.rand(X.shape[0], X.shape[0])
    a = np.random.uniform(0, 1)
    b = np.random.uniform(0, 1)
    
    #iter
    for iter in range(maxiter):
        for k in range(3):
            Gshape=G.shape
            #Kshape=K.shape
            #Xshape=X.shape
            G=unfold(G,k)
            #X=unfold(X,k)
            #K=unfold(K,k)
            print(K.shape,X.shape,G.shape)
            #G = conj_grad_G(G, S, R, M, X, K,n,r)
            #G = fold(G, Gshape, k)
            #K = fold(K, Kshape, k)
            #X = fold(X, Xshape, k)
            
            S, R, M = conj_grad_S_R_M(G, S, R, M, X, K,Q,H,a,b,n ,r)
            Q, H = conj_grad_Q_H(S,R,M,Q1, H, a, b)
            a, b = conj_grad_a_b(Q1, H,S ,a, b)
        if Res()<thers:
            break
    return G,S,R,M,Q,H,a,b

In [42]:
rank=32
thers=0.001
maxiter=5
n=0.1
r=0.1
G,S,R,M,Q,H,a,b=Star(X0,X,rank,n,r,thers,maxiter)

t1 = np.einsum('ijh, ki -> kjh', G, S)
t2 = np.einsum('ijh, kj -> ikh', t1, R)
t3 = np.einsum('ijh, kh -> ijk', t2, M)


    #rank=[50,50,50]
    #core, factors = tucker(X, rank)
    #G = core
    #S = factors[0]
    #R = factors[1]
    #M = factors[2]

(32, 32, 32)
(32, 32, 32) (32, 32) (32, 32) (32, 32)
(32, 32, 32) (32, 32, 32) (32, 1024)
bbbb
(32, 32)
[[0.55033835 0.78303102 0.8048355  ... 0.12029634 0.46576104 0.88650222]
 [0.48733443 0.14379574 0.2557504  ... 0.4660495  0.40415332 0.06777642]
 [0.42280406 0.3916341  0.74136166 ... 0.84630407 0.12937492 0.63919134]
 ...
 [0.42900777 0.36049658 0.16208908 ... 0.83757334 0.53564311 0.07169365]
 [0.76968177 0.797534   0.88048839 ... 0.12516005 0.00622935 0.26336359]
 [0.56128398 0.33353888 0.17133544 ... 0.65511511 0.45609828 0.8790776 ]]


AxisError: source: axis 2 is out of bounds for array of dimension 2